<a href="https://colab.research.google.com/github/rjpaxtondata/Deep_Learning_Charity_Data/blob/main/Starter_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import io



In [ ]:
#use google collab import data option
from google.colab import files
uploaded = files.upload()

Saving charity_data.csv to charity_data (1).csv


In [ ]:
#Read the csv using the google collab pd.read
import io
df = pd.read_csv(io.StringIO(uploaded['charity_data.csv'].decode('utf-8')), sep = ',')
df

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

df = df.drop(['EIN', 'NAME'], axis=1)

In [ ]:
# Determine the number of unique values in each column.
df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
app_type = df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type[app_type < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
class_df = df['CLASSIFICATION'].value_counts()
class_df

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C6100        1
C1732        1
C4120        1
C1728        1
C2600        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_df[class_df > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1237        9
C1235        9
C7210        7
C2400        6
C4100        6
C1720        6
C1600        5
C1257        5
C2710        3
C1260        3
C0           3
C1234        2
C1256        2
C3200        2
C1267        2
C1246        2
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_df[class_df < 1883].index)

# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies = pd.get_dummies(df)

In [ ]:
# Split our preprocessed data into our features and target arrays
y = dummies['IS_SUCCESSFUL']
x = dummies.drop(columns=['IS_SUCCESSFUL'])
# Split the preprocessed data into a training and testing dataset
xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=123)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(xtrain)

# Scale the data
X_train_scaled = X_scaler.transform(xtrain)
X_test_scaled = X_scaler.transform(xtest)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_layers = len(x.columns)
#layer1=50
#layer2=25
nn = tf.keras.models.Sequential()


# First hidden layer (units=layer1)
nn.add(tf.keras.layers.Dense(units=50, input_dim=input_layers, activation='relu'))

# Second hidden layer (units=layer2)
nn.add(tf.keras.layers.Dense(units=25, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                2200      
                                                                 
 dense_1 (Dense)             (None, 25)                1275      
                                                                 
 dense_2 (Dense)             (None, 1)                 26        
                                                                 
Total params: 3,501
Trainable params: 3,501
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
model_fit = nn.fit(X_train_scaled, ytrain, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5756 - accuracy: 0.7173
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5561 - accuracy: 0.7294
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5522 - accuracy: 0.7302
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5514 - accuracy: 0.7314
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5497 - accuracy: 0.7316
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5493 - accuracy: 0.7317
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5480 - accuracy: 0.7320
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5475 - accuracy: 0.7324
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5471 - accuracy: 0.7346
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5462 - accura

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, ytest,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5576 - accuracy: 0.7292 - 443ms/epoch - 2ms/step
Loss: 0.557610809803009, Accuracy: 0.7292128205299377


In [ ]:
# Procedure still running after 1.5hrs 
#Find the best hyperparameter options
# def create_model(hp):
#   nn_model = tf.keras.models.Sequential()
#   input_features = len(X_train_scaled[0])
  
#   #Allow kerastuner to decide which activation function to use in hidden layers
#   activation = hp.Choice('activation',['relu','tanh','sigmoid'])

#   # Allow kerastuner to decide number of neurons in first layer
#   nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
#         min_value=60,
#         max_value=200,
#         step=15), 
#         activation=activation, 
#         input_dim=input_features))                                    
  
#   ## Allow kerastuner to decide number of hidden layers and neurons in hidden layers
#   for i in range(hp.Int('num_layers', 1, 6)):
#         nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
#             min_value=1,
#             max_value=10,
#             step=2),
#             activation=activation))
  
#   ## Standard output layer with 1 sigmoid unit
#   nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

#   # Compile the model
#   nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
#   return nn_model



In [ ]:
# !pip install keras-tuner
# # Import the kerastuner library
# import kerastuner as kt

# tuner = kt.Hyperband(
#     create_model,
#     objective="val_accuracy",
#     max_epochs=100,
#     hyperband_iterations=2)

In [ ]:
# # Run the kerastuner search for best hyperparameters
# tuner.search(X_train_scaled,ytrain,epochs=100,validation_data=(X_test_scaled,ytest))

In [ ]:
#Obtain accuracy higher than 75%

In [ ]:
# Define the model#1 - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_layers = len(x.columns)
#layer1=50
#layer2=25
nn1 = tf.keras.models.Sequential()


# First hidden layer (units=layer1)
nn1.add(tf.keras.layers.Dense(units=250, input_dim=input_layers, activation='relu'))

# Second hidden layer (units=layer2)
nn1.add(tf.keras.layers.Dense(units=100, activation='relu'))

# third hidden layer (units=layer2)
nn1.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 250)               11000     
                                                                 
 dense_8 (Dense)             (None, 100)               25100     
                                                                 
 dense_9 (Dense)             (None, 50)                5050      
                                                                 
 dense_10 (Dense)            (None, 1)                 51        
                                                                 
Total params: 41,201
Trainable params: 41,201
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model#1
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model#1
model_fit = nn1.fit(X_train_scaled, ytrain, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5652 - accuracy: 0.7234
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5546 - accuracy: 0.7304
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5523 - accuracy: 0.7301
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5507 - accuracy: 0.7320
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5490 - accuracy: 0.7316
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5487 - accuracy: 0.7323
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5477 - accuracy: 0.7329
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5464 - accuracy: 0.7345
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5455 - accuracy: 0.7339
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5465 - accura

In [ ]:
# Evaluate the model using the test data - model#1
model_loss, model_accuracy = nn1.evaluate(X_test_scaled, ytest,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5900 - accuracy: 0.7303 - 473ms/epoch - 2ms/step
Loss: 0.5900149941444397, Accuracy: 0.7302623987197876


In [ ]:
#Model #2
input_layers = len(x.columns)
#layer1=50
#layer2=25
nn2 = tf.keras.models.Sequential()


# First hidden layer (units=layer1)
nn2.add(tf.keras.layers.Dense(units=500, input_dim=input_layers, activation='relu'))

# Second hidden layer (units=layer2)
nn2.add(tf.keras.layers.Dense(units=250, activation='relu'))

# third hidden layer (units=layer2)
nn2.add(tf.keras.layers.Dense(units=100, activation='relu'))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 500)               22000     
                                                                 
 dense_12 (Dense)            (None, 250)               125250    
                                                                 
 dense_13 (Dense)            (None, 100)               25100     
                                                                 
 dense_14 (Dense)            (None, 1)                 101       
                                                                 
Total params: 172,451
Trainable params: 172,451
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model#3
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model#3
model_fit = nn2.fit(X_train_scaled, ytrain, epochs=100)

Epoch 1/100
804/804 [==============================] - 5s 5ms/step - loss: 0.5688 - accuracy: 0.7233
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5564 - accuracy: 0.7280
Epoch 3/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5544 - accuracy: 0.7304
Epoch 4/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5524 - accuracy: 0.7297
Epoch 5/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5504 - accuracy: 0.7316
Epoch 6/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5490 - accuracy: 0.7333
Epoch 7/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5487 - accuracy: 0.7332
Epoch 8/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5477 - accuracy: 0.7337
Epoch 9/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5468 - accuracy: 0.7341
Epoch 10/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5461 - accura

In [ ]:
# Evaluate the model using the test data - Model#2
model_loss, model_accuracy = nn2.evaluate(X_test_scaled, ytest,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5829 - accuracy: 0.7284 - 594ms/epoch - 2ms/step
Loss: 0.5828978419303894, Accuracy: 0.728396475315094


In [ ]:
# Define the model#3 - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_layers = len(x.columns)
#layer1=50
#layer2=25
nn3 = tf.keras.models.Sequential()


# First hidden layer (units=layer1)
nn3.add(tf.keras.layers.Dense(units=100, input_dim=input_layers, activation='relu'))

# Second hidden layer (units=layer2)
nn3.add(tf.keras.layers.Dense(units=100, activation='sigmoid'))

# third hidden layer (units=layer2)
nn3.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 100)               4400      
                                                                 
 dense_16 (Dense)            (None, 100)               10100     
                                                                 
 dense_17 (Dense)            (None, 50)                5050      
                                                                 
 dense_18 (Dense)            (None, 1)                 51        
                                                                 
Total params: 19,601
Trainable params: 19,601
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model#3
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model - Model#3
model_fit = nn3.fit(X_train_scaled, ytrain, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5728 - accuracy: 0.7183
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5564 - accuracy: 0.7286
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5535 - accuracy: 0.7288
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5510 - accuracy: 0.7321
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5505 - accuracy: 0.7324
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5493 - accuracy: 0.7307
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5484 - accuracy: 0.7330
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5480 - accuracy: 0.7321
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5473 - accuracy: 0.7328
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5467 - accura

In [ ]:
# Evaluate the model using the test data - Model #3
model_loss, model_accuracy = nn3.evaluate(X_test_scaled, ytest,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5821 - accuracy: 0.7290 - 442ms/epoch - 2ms/step
Loss: 0.582116425037384, Accuracy: 0.7289795875549316


In [ ]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")